# Построение рекомендательной системы книг на основе датасета goodbooks-10k с созданием визульного протатипа на основе streamlib

## Небольшая предобработка данных

In [5]:
import pickle
import joblib

import pandas as pd
import numpy as np
import scipy.sparse as sparse

from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

In [6]:
# Прочитаем исходные данные
ratings = pd.read_csv('app/data/ratings.csv')
books = pd.read_csv('app/data/books.csv')
tags = pd.read_csv('app/data/tags.csv')
book_tags = pd.read_csv('app/data/book_tags.csv')

In [7]:
tags.head()

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-


В файле tags.csv очень много неинформативных тегов.
В нашем случае теги сильно влияют на качество модели. Поэтому мы заранее произвели их предобработку:

Выбрали около 500 наиболее популярных тегов.
Сгруппировали оставшиеся теги.

In [8]:
tags = pd.read_csv('app/data/tags_cleaned.csv')

В датафреймах tags и books есть два идентификатора: goodreads_book_id — от сервиса Goodreads и book_id, который привязан к нашей таблице. Создадим словарь, с помощью которого сможем находить book_id книги по goodreads_book_id.

In [9]:
mapper = dict(zip(books.goodreads_book_id,books.book_id))

In [10]:
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])

Для работы с моделями в библиотеке LightFM необходимо создать разрежённые матрицы. Мы будем хранить данные в самом простом формате — COO.

In [11]:
ratings_coo = sparse.coo_matrix((ratings.rating,(ratings.user_id,ratings.book_id)))
feature_ratings  = sparse.coo_matrix(([1]*len(book_tags),(book_tags.id,book_tags.tag_id)))

## Построение модели

In [12]:
# число потоков процессора (зависит от того, на какой машине запускаете)
NUM_THREADS = 8 

#число параметров вектора 
NUM_COMPONENTS = 60 

#число эпох обучения
NUM_EPOCHS = 10

#зерно датчика случайных чисел
RANDOM_STATE = 42

``` python

# число потоков процессора (зависит от того, на какой машине запускаете)
NUM_THREADS = 8 

#число параметров вектора 
NUM_COMPONENTS = 60 

#число эпох обучения
NUM_EPOCHS = 10

#зерно датчика случайных чисел
RANDOM_STATE = 42

#Разбиваем датасет на обучающую и тестовую выборки
train, test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=RANDOM_STATE)

#Создаём модель
model = LightFM(
    learning_rate=0.05, #темп (скорость) обучения
    loss='warp', #loss-функция
    no_components=NUM_COMPONENTS,#размерность вектора признаков
    random_state=RANDON_STATE #генератор случайных чисел
)

#Обучаем модель
model = model.fit(
    train, #обучающая выборка
    epochs=NUM_EPOCHS, #количество эпох обучения
    num_threads=NUM_THREADS, #количество потоков процессора
    item_features=feature_ratings #признаки товаров (рейтинги книг)
)

#Тестируем нашу модель
precision_score = precision_at_k(
    model, #модель
    test, #тестовая выборка
    num_threads=NUM_THREADS, #количество потоков процессора
    k=10, #количество предложений
    item_features=feature_ratings #признаки товаров
).mean() #усредняем результаты
 
recall_score = recall_at_k(
    model, #модель
    test, #тестовая выборка
    num_threads=NUM_THREADS, #количество потоков процессора
    k=10, #количество предложений
    item_features=feature_ratings #признаки товаров
).mean() #усредняем результаты

print(recall_score, precision_score)

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

```

> Из-за большого размера обучающей выборки процесс обучения модели LightFM может занять вплоть до нескольких часов в зависимости от ОС и характеристик компьютера. Чтобы не тратить время и сосредоточиться на следующих этапах, отдельно обучим модель и сохраним ее в формат pickle.

In [13]:
with open('app/model.pkl', 'rb') as file:
    model = pickle.load(file)

## Тестирование

In [14]:
#Разбиваем датасет на обучающую и тестовую выборки
train, test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=RANDOM_STATE)
#Тестируем нашу модель
precision_score = precision_at_k(
    model, #модель
    test, #тестовая выборка
    num_threads=NUM_THREADS, #количество потоков процессора
    k=10, #количество предложений
    item_features=feature_ratings #признаки товаров
).mean() #усредняем результаты
 
recall_score = recall_at_k(
    model, #модель
    test, #тестовая выборка
    num_threads=NUM_THREADS, #количество потоков процессора
    k=10, #количество предложений
    item_features=feature_ratings #признаки товаров
).mean() #усредняем результаты

print(recall_score, precision_score)

0.042941267380755835 0.0925424


Результаты получаются неидеальными, однако для тестирования функциональности такого baseline будет достаточно.

## Добавление эмбедддингов

In [15]:
# Извлекаем эмбеддинги
item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

print(item_biases.shape, item_embeddings.shape)
## (10001,) (10001, 60)

(10001,) (10001, 60)


Будем использовать метод ближайших соседей (approximate k-nn), который реализован в библиотеке NMSLIB.

В основе метода лежит идея о том, что вместо того чтобы перебирать все векторы, мы можем очень быстро обходить граф, каждая вершина которого соответсвует эмбеддингу книги.

In [16]:
import nmslib

#Создаём граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')
 
#Начинаем добавлять книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

#Вспомогательная функция для поиска по графу
def nearest_books_nms(book_id, index, n=10):
    nn = index.knnQuery(item_embeddings[book_id], k=n)
    return nn


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


Давайте протестируем работу поиска рекомендаций на уровне интуиции.

## АНАЛИЗ РЕКОМЕНДАЦИЙ ПОСТРОЕННОЙ МОДЕЛИ

Теперь, когда у нас есть функция для поиска десяти наиболее похожих книг, давайте попробуем написать рекомендации к какой-нибудь книге, например к роману «1984» Джорджа Оруэлла. Для этого воспользуемся частичным поиском по неполным совпадениям в столбце title. Уберём чувствительность к регистру, приведя оригинальные названия к нижнему регистру.

In [17]:
#Отфильтруем только те книги, в которых названии встречается подстрока "1984"
books[books['title'].apply(lambda x: x.lower().find('1984')) >= 0]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
12,13,5470,5470,153313,995,451524934,9.780452e+12,"George Orwell, Erich Fromm, Celâl Üster",1949.0,Nineteen Eighty-Four,...,1956832,2053394,45518,41845,86425,324874,692021,908229,https://images.gr-assets.com/books/1348990566m...,https://images.gr-assets.com/books/1348990566s...
845,846,5472,5472,2966408,51,151010269,9.780151e+12,"George Orwell, Christopher Hitchens",1950.0,Animal Farm & 1984,...,116197,118761,1293,1212,3276,16511,40583,57179,https://images.gr-assets.com/books/1327959366m...,https://images.gr-assets.com/books/1327959366s...
9795,9796,201145,201145,2563528,25,64440508,9.780064e+12,"Else Holmelund Minarik, Maurice Sendak",1968.0,A Kiss for Little Bear,...,11063,11604,126,87,284,1898,3053,6282,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


In [18]:
#Вызываем функцию для поиска ближайших соседей
nearest_books_nms(846, nms_idx)

(array([846,  14,  55,  48, 809,  13, 903, 529, 271, 173], dtype=int32),
 array([0.        , 0.03706956, 0.04132241, 0.06263614, 0.07201874,
        0.07343107, 0.09369212, 0.09787107, 0.10064054, 0.10802871],
       dtype=float32))

Как мы говорили ранее, наша функция возвращает кортеж из двух массивов: numpy-вектор идентификаторов и numpy-вектор мер схожести (расстояний) между заданной книгой и её ближайшими соседями.

Пока что нас интересуют только идентификаторы— выделим их в отдельную переменную:

In [19]:
#Выделяем идентификаторы рекомендованных книг
nbm = nearest_books_nms(846, nms_idx)[0]
nbm
## array([846,  14,  55,  48, 809,  13, 903, 529, 271, 173])

array([846,  14,  55,  48, 809,  13, 903, 529, 271, 173], dtype=int32)

Обратим внимание, что ближайшей к заданной книге всегда будет она сама, поэтому идентификатор этой книги всегда будет первым в списке рекомендаций. Пока что мы закроем на это глаза, однако в дальнейшем при разработке прототипа приложения мы исправим этот момент.

In [20]:
#Посмотрим на авторов и названия рекомендованных книг
books[books.book_id.isin(nbm)][['authors', 'title']]

,authors,title
12,"George Orwell, Erich Fromm, Celâl Üster",1984
13,George Orwell,Animal Farm
47,Ray Bradbury,Fahrenheit 451
54,Aldous Huxley,Brave New World
172,Anthony Burgess,A Clockwork Orange
270,Daniel Keyes,Flowers for Algernon
528,"Jonathan Swift, Robert DeMaria Jr.",Gulliver's Travels
808,"Aldous Huxley, Christopher Hitchens",Brave New World / Brave New World Revisited
845,"George Orwell, Christopher Hitchens",Animal Farm / 1984
902,Ayn Rand,Anthem


Как можно увидеть, в результатах есть действительно похожие книги. Первыми идут отдельные книги — «1984» и «Скотный двор» ("Animal Farm"). Далее следуют «451 градус по Фаренгейту» Рэя Брэдбери ("Fahrenheit 451") и «О дивный новый мир» Олдоса Хаксли ("Brave New World"), которые также являются антиутопиями.

Сохраним полученные эмбеддинги — они пригодятся в дальнейшем при реализации прототипа:

In [21]:
with open('app/item_embeddings.pkl', 'wb') as file:
    pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)


Теперь создадим визуальный прототип нашей модели -> app.py